# buy-and-hold

buy, then never ever sell, until the end date :)

In [1]:
# other imports
import pandas as pd
import datetime

# project imports
import pinkfish as pf

Some global data

In [2]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Timeseries

In [3]:
# fetch timeseries, select, finalize
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=True, pad=False)
ts, start = pf.finalize_timeseries(ts, start)

# create tradelog and daily balance objects
tlog = pf.TradeLog()
dbal = pf.DailyBal()

Algorithm

In [4]:
# init tradelog
tlog.initialize(capital)

# loop through timeseries
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # buy
    if tlog.shares == 0:
        tlog.enter_trade(date, row.close)
    # sell
    elif end_flag:
        tlog.exit_trade(date, row.close)

    # record daily balance
    dbal.append(date, row.high, row.low, row.close, tlog.shares)

Retrieve logs

In [5]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [6]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,symbol
0,1993-01-29,26.299288,2020-06-08,323.200012,296.900724,112822.275276,380,112822.275276,


Get stats

In [7]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [8]:
pf.summary(stats)

,strategy
annual_return_rate,9.601597
max_closed_out_drawdown,-55.181901
drawdown_annualized_return,-5.747158
drawdown_recovery,-1.415498
best_month,23.613971
worst_month,-30.985202
sharpe_ratio,0.580277
sortino_ratio,0.734416
monthly_std,4.488608
